In [11]:
import os
import io
import re
import collections as col

"""
What does this do?
Cut out sequence for unknown/black segment from phased assembly contigs
to align against GRCh38, and to perform pairwise sequence alignment checking
for broad homology between unknown/black segments coming from different assemblies.
Currently, not used or mentioned in the manuscript.
"""

segments = """
bng_row_index   sample population platform assembly_type haplotype  \
110             45  HG02011        AFR      CLR        phased        H1   
189             65  NA19238        AFR      CLR        phased        H2   

    bng_contig_id  component_start  component_end orientation  \
110        h1-649               -1         160528      direct   
189       h2-1014             7554         191560      direct   

     component_length             contig_id variant_type  \
110           2389451  cluster16_contig_129   no_variant   
189           2416853    cluster2_contig_16   no_variant   

    variant_segment_chain  variant_start  variant_end segment_color  \
110      no_segment_chain             -1           -1         black   
189      no_segment_chain             -1           -1         black   

     segment_start  segment_end  segment_support  
110          53703        99375                2  
189          41551        88156                2
"""

path = '/home/local/work/data/hgsvc/roi/dotplots_chr3/chr3_ctg_fasta'
fasta_files = [
    'HG02011_hgsvc_pbsq2-clr_1000-flye.h1-un.arrow-p1.ctg3q29.fasta',
    'NA19238_hgsvc_pbsq2-clr_1000-flye.h2-un.arrow-p1.ctg3q29.fasta'
]

def count_motifs(sequence):
    
    forward = re.findall('CTTAAG', sequence.upper())
    reverse = re.findall('GAATTC', sequence.upper())
    return len(forward), len(reverse)


seq_buffer = io.StringIO()
contig_buffer = ''
buffered_contig = ''
record = False
for f in fasta_files:
    fpath = os.path.join(path, f)
    with open(fpath, 'r') as fasta:
        for line in fasta:
            if line.startswith('>'):
                contig = line.strip('>').strip()
                if contig in ['cluster16_contig_129', 'cluster2_contig_16']:
                    record = True
                if contig_buffer:
                    if buffered_contig == 'cluster16_contig_129':
                        contig_buffer = contig_buffer[53703:99375]
                        print(buffered_contig)
                        print(count_motifs(contig_buffer))
                        seq_buffer.write('>{}_black'.format(buffered_contig) + '\n')
                        seq_buffer.write(contig_buffer + '\n')
                    elif buffered_contig == 'cluster2_contig_16':
                        contig_buffer = contig_buffer[41551:88156]
                        print(buffered_contig)
                        print(count_motifs(contig_buffer))
                        seq_buffer.write('>{}_black'.format(buffered_contig) + '\n')
                        seq_buffer.write(contig_buffer + '\n')
                    else:
                        pass
                contig_buffer = ''
                if record:
                    buffered_contig = contig
                continue
            if record:
                contig_buffer += line.strip()
if contig_buffer:
    if buffered_contig == 'cluster16_contig_129':
        contig_buffer = contig_buffer[53703:99375]
        seq_buffer.write('>{}_black'.format(buffered_contig) + '\n')
        seq_buffer.write(contig_buffer + '\n')
    elif buffered_contig == 'cluster2_contig_16':
        contig_buffer = contig_buffer[41551:88156]
        print(buffered_contig)
        print(count_motifs(contig_buffer))
        seq_buffer.write('>{}_black'.format(buffered_contig) + '\n')
        seq_buffer.write(contig_buffer + '\n')

with open(os.path.join(path, 'HG02011-H1-CLR_NA19238-H2-CLR_black.fasta'), 'w') as fasta:
    _ = fasta.write(seq_buffer.getvalue())

cluster16_contig_129
(3, 13)
cluster2_contig_16
(15, 19)
